In [1]:
import sqlite3
import pandas as pd

url_dict = {
'airports' : ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat',["Airport ID", "Name","City",'Country','IATA','ICAO','Latitude','Longitude','Altitude',
                                                                                                 'Timezone','DST','Tz database timezone','Type','Source']),
'airlines' : ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat',['Airline ID','Name','Alias','IATA','ICAO','Callsign','Country','Active']),
'routes' : ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat',['Airline','Airline ID','Source airport','Source airport ID','Destination airport',
                                                                                             'Destination airport ID','Codeshare','Stops','Equipment']),
'planes' : ('https://raw.githubusercontent.com/jpatokal/openflights/master/data/planes.dat',['Name','IATA code','ICAO code'])
}

conn = sqlite3.connect("openflights.db")
cursor = conn.cursor()

for db_name, (url,columns) in url_dict.items():
    df = pd.read_csv(url,names=columns)
    df.columns = df.columns.str.lower().str.replace(" ", "_")
    print(df)
    df.to_sql(db_name,conn,if_exists='replace',index = False),

      airport_id                                         name          city  \
0              1                               Goroka Airport        Goroka   
1              2                               Madang Airport        Madang   
2              3                 Mount Hagen Kagamuga Airport   Mount Hagen   
3              4                               Nadzab Airport        Nadzab   
4              5  Port Moresby Jacksons International Airport  Port Moresby   
...          ...                                          ...           ...   
7693       14106                          Rogachyovo Air Base        Belaya   
7694       14107                        Ulan-Ude East Airport      Ulan Ude   
7695       14108                         Krechevitsy Air Base      Novgorod   
7696       14109                  Desierto de Atacama Airport       Copiapo   
7697       14110                           Melitopol Air Base     Melitopol   

               country iata  icao   latitude   long

In [2]:
# Use multiline queries to make it easier to read and build
query = """
SELECT *
FROM airports
WHERE airport_id = 1
"""

df = pd.read_sql_query(query,conn)
df

,airport_id,name,city,country,iata,icao,latitude,longitude,altitude,timezone,dst,tz_database_timezone,type,source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.08169,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports


<h1>SQL Quiz</h1>
<h3>Question 1<h3>

In [3]:
Q1 = """
SELECT *
FROM airports
LIMIT 5
"""

df1 = pd.read_sql_query(Q1,conn)
df1

,airport_id,name,city,country,iata,icao,latitude,longitude,altitude,timezone,dst,tz_database_timezone,type,source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports


<h3>Question 2</h3>

In [4]:
Q2 = """
SELECT name, city, country
FROM airports
WHERE country = 'United States'
"""

df2 = pd.read_sql_query(Q2,conn)
df2

,name,city,country
0,Barter Island LRRS Airport,Barter Island,United States
1,Wainwright Air Station,Fort Wainwright,United States
2,Cape Lisburne LRRS Airport,Cape Lisburne,United States
3,Point Lay LRRS Airport,Point Lay,United States
4,Hilo International Airport,Hilo,United States
...,...,...,...
1507,Camp Pendleton MCAS (Munn Field) Airport,Oceanside,United States
1508,Vidalia Regional Airport,Vidalia,United States
1509,Granbury Regional Airport,Granbury,United States
1510,Oswego County Airport,Fulton,United States


<h3>Question 3</h3>

In [5]:
Q3 = """
SELECT
    al.name,
    al.airline_id,
    COUNT(DISTINCT rt.source_airport) AS count
FROM airlines al
INNER JOIN routes rt ON rt.airline_id = al.airline_id
WHERE al.active = 'Y'
GROUP BY name, al.airline_id
HAVING COUNT(DISTINCT rt.source_airport) >= 3
ORDER BY
    COUNT(DISTINCT rt.source_airport) DESC
"""

df3 = pd.read_sql_query(Q3,conn)
df3

,name,airline_id,count
0,American Airlines,24,429
1,United Airlines,5209,426
2,Air France,137,378
3,KLM Royal Dutch Airlines,3090,360
4,US Airways,5265,348
...,...,...,...
496,Kam Air,3097,3
497,Omni Air International,3781,3
498,PB Air,3835,3
499,Rossiya-Russian Airlines,641,3


<h3>Question 4<h3>

In [6]:
Q4 = """
SELECT
    ap.airport_id,
    ap.name,
    ap.city,
    ap.country,
    ap.altitude,
    COUNT(rt.source_airport_id) AS total_number_of_routes
FROM airports ap
INNER JOIN routes rt ON rt.source_airport_id = ap.airport_id
GROUP BY
    ap.airport_id,
    ap.name,
    ap.city,
    ap.country,
    ap.altitude
ORDER BY
    total_number_of_routes DESC,
    ap.altitude DESC
"""

df4 = pd.read_sql_query(Q4,conn)
df4

,airport_id,name,city,country,altitude,total_number_of_routes
0,3682,Hartsfield Jackson Atlanta International Airport,Atlanta,United States,1026,915
1,3830,Chicago O'Hare International Airport,Chicago,United States,672,558
2,3364,Beijing Capital International Airport,Beijing,China,116,535
3,507,London Heathrow Airport,London,United Kingdom,83,527
4,1382,Charles de Gaulle International Airport,Paris,France,392,524
...,...,...,...,...,...,...
3206,7136,Elfin Cove Seaplane Base,Elfin Cove,United States,0,1
3207,7148,Hydaburg Seaplane Base,Hydaburg,United States,0,1
3208,7309,Charlotte Amalie Harbor Seaplane Base,Charlotte Amalie,Virgin Islands,0,1
3209,7558,Ulaangom Airport,Ulaangom,Mongolia,0,1


<h3>Question 5</h3>

In [7]:
Q5 = """
SELECT
    source_airport,
    COUNT(source_airport_id) AS count_of_routes,
    DENSE_RANK() OVER (ORDER BY COUNT(source_airport_id) DESC) AS rank
FROM routes
GROUP BY
    source_airport
HAVING
    count_of_routes >= 5
ORDER BY rank
"""

df5 = pd.read_sql_query(Q5, conn)
df5

,source_airport,count_of_routes,rank
0,ATL,915,1
1,ORD,558,2
2,PEK,535,3
3,LHR,527,4
4,CDG,524,5
...,...,...,...
1504,ARI,5,213
1505,APW,5,213
1506,AOR,5,213
1507,AOJ,5,213


<h3>Question 6</h3>

In [8]:
Q6 = """
SELECT
    al.name AS airline_name,
    al.country AS airline_country,
    src.name AS source_airport_name,
    src.city AS source_airport_city,
    dst.name AS destination_airport_name,
    dst.city AS destination_city
FROM airlines al
INNER JOIN routes rt ON rt.airline_id = al.airline_id
INNER JOIN airports AS src ON src.airport_id = rt.source_airport_id
INNER JOIN airports AS dst ON dst.airport_id = rt.destination_airport_id
WHERE
    al.country = src.country AND
    al.country = dst.country
"""

df6 = pd.read_sql_query(Q6, conn)
df6

,airline_name,airline_country,source_airport_name,source_airport_city,destination_airport_name,destination_city
0,Star Peru (2I),Peru,Coronel FAP Alfredo Mendivil Duarte Airport,Ayacucho,Jorge Chávez International Airport,Lima
1,Star Peru (2I),Peru,Alejandro Velasco Astete International Airport,Cuzco,Jorge Chávez International Airport,Lima
2,Star Peru (2I),Peru,Alejandro Velasco Astete International Airport,Cuzco,Padre Aldamiz International Airport,Puerto Maldonado
3,Star Peru (2I),Peru,Alferez Fap David Figueroa Fernandini Airport,Huánuco,Jorge Chávez International Airport,Lima
4,Star Peru (2I),Peru,Coronel FAP Francisco Secada Vignetta Internat...,Iquitos,Cap FAP David Abenzur Rengifo International Ai...,Pucallpa
...,...,...,...,...,...,...
25293,Regional Express,Australia,Wagga Wagga City Airport,Wagga Wagga,Melbourne International Airport,Melbourne
25294,Regional Express,Australia,Wagga Wagga City Airport,Wagga Wagga,Sydney Kingsford Smith International Airport,Sydney
25295,Regional Express,Australia,Winton Airport,Winton,Longreach Airport,Longreach
25296,Regional Express,Australia,Winton Airport,Winton,Townsville Airport,Townsville


<h3>Question 7</h3>

In [9]:
Q7 = """
SELECT
    ap.name,
    ap.city,
    ap.country
FROM airports ap
LEFT JOIN routes rt ON rt.source_airport_id = ap.airport_id OR rt.destination_airport_id = ap.airport_id
WHERE
    rt.source_airport_id IS NULL AND
    rt.destination_airport_id IS NULL
"""

df7 = pd.read_sql_query(Q7, conn)
df7

,name,city,country
0,Hornafjörður Airport,Hofn,Iceland
1,Húsavík Airport,Husavik,Iceland
2,Patreksfjörður Airport,Patreksfjordur,Iceland
3,Siglufjörður Airport,Siglufjordur,Iceland
4,Vestmannaeyjar Airport,Vestmannaeyjar,Iceland
...,...,...,...
4472,Rogachyovo Air Base,Belaya,Russia
4473,Ulan-Ude East Airport,Ulan Ude,Russia
4474,Krechevitsy Air Base,Novgorod,Russia
4475,Desierto de Atacama Airport,Copiapo,Chile


<h3>Question 8</h3>

In [10]:
Q8 = """
SELECT
    src.name AS source_airport,
    GROUP_CONCAT(dst.name, ', ') AS destination_airport
FROM (
    SELECT DISTINCT rt.source_airport_id, rt.destination_airport_id
    FROM routes rt
) routes
INNER JOIN airports AS src ON src.airport_id = routes.source_airport_id
INNER JOIN airports AS dst ON dst.airport_id = routes.destination_airport_id
GROUP BY
    src.airport_id
HAVING
    COUNT(dst.airport_id) > 1
"""

df8 = pd.read_sql_query(Q8, conn)
df8

,source_airport,destination_airport
0,Goroka Airport,"Mount Hagen Kagamuga Airport, Nadzab Airport, ..."
1,Madang Airport,"Goroka Airport, Mount Hagen Kagamuga Airport, ..."
2,Mount Hagen Kagamuga Airport,"Goroka Airport, Nadzab Airport, Madang Airport..."
3,Nadzab Airport,"Daru Airport, Goroka Airport, Mount Hagen Kaga..."
4,Port Moresby Jacksons International Airport,"Brisbane International Airport, Daru Airport, ..."
...,...,...
2390,Erenhot Saiwusu International Airport,"Baita International Airport, Beijing Capital I..."
2391,Hamad International Airport,"Chhatrapati Shivaji International Airport, Coc..."
2392,Beni Airport,"Bunia Airport, Goma International Airport"
2393,Dharavandhoo Airport,"Hanimaadhoo Airport, Malé International Airport"


<h3>Question 9</h3>

In [11]:
Q9 = """
SELECT *
FROM airlines
WHERE airline_id = '21270'
"""
# Showing that the "active" column previously said "Y"

df9 = pd.read_sql_query(Q9, conn)
df9

,airline_id,name,alias,iata,icao,callsign,country,active
0,21270,Air Carnival,None,2S,\N,None,India,Y


In [12]:
Q9_update = """
UPDATE airlines
SET active = 'N'
WHERE airline_id = '21270'
"""

df9_update = pd.read_sql_query(Q9_update, conn)
df9_update
# You can't read a SQL update

TypeError: 'NoneType' object is not iterable

In [13]:
Q9_view = """
SELECT *
FROM airlines
WHERE airline_id = '21270'
"""

df9_view = pd.read_sql_query(Q9_view, conn)
df9_view

,airline_id,name,alias,iata,icao,callsign,country,active
0,21270,Air Carnival,None,2S,\N,None,India,N


<h3>Question 10</h3>

In [14]:
Q10 = """
SELECT *
FROM routes
WHERE stops > 0
"""

# Show that there are routes with more than 0 stops
df10 = pd.read_sql_query(Q10, conn)
df10

,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment
0,5T,1623,YRT,132,YEK,50,None,1,ATR
1,AC,330,ABJ,253,BRU,302,None,1,333
2,AC,330,YVR,156,YBL,30,None,1,BEH
3,CU,1936,FCO,1555,HAV,1909,None,1,767
4,FL,1316,HOU,3566,SAT,3621,None,1,735
5,FL,1316,MCO,3878,HOU,3566,None,1,73W
6,FL,1316,MCO,3878,ORF,3611,None,1,717
7,SK,4319,ARN,737,GEV,715,None,1,ATP
8,WN,4547,BOS,3448,MCO,3878,None,1,73W
9,WN,4547,MCO,3878,BOS,3448,None,1,73W


In [15]:
cursor.executescript("""
                     BEGIN TRANSACTION;
                     DELETE FROM routes WHERE stops > 0;
                     COMMIT;
                     """)

cursor.execute("SELECT * FROM routes WHERE stops > 0;")

# I didn't use cursor in the previous question because I'm not sure if we
# were supposed to commit the update. But, reading the update using pandas
# would make the changes and execute the update.

In [16]:
Q10_view = """
SELECT *
FROM routes
WHERE stops > 0
"""

df10_view = pd.read_sql_query(Q10_view, conn)
df10_view

,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment


<h2>Additional Tasks</h2>
<h3>Build Project Experience on Resume</h3>
<ul>Conduct exploratory data analysis using Pandas and Numpy to discover insights through descriptive statistics and data visualizations</ul>
<ul>Create interactive, shareable data visualizations of data analysis and achine learning models using Plotly and Streamlit</ul>

<h3>Datasets</h3>
<ol>https://www.kaggle.com/datasets/waqi786/most-used-beauty-cosmetics-products-in-the-world - Focus on cosmetic products</ol>
<ol>https://www.kaggle.com/datasets/catherinerasgaitis/mxmh-survey-results - Music and Mental Health</ol>
<ol>https://www.kaggle.com/datasets/ankitkr60/advertisement-and-sales-data-for-analysis - Advertisement and Sales</ol>